In [13]:

import math
import re
import random
import nltk
from bs4 import BeautifulSoup
random.seed(0)

make_random_mask = False
exact_answers = []
not_include_answer = []
span_len = 16
WINDOWING=False

def web_parse(text):
    soup = BeautifulSoup(text)
    text = soup.get_text()
    return text

def find_random_idx(ans_start_tok, answer_end_tok,txt):

    max_bound = max(len(re.split(' ',txt))-(answer_end_tok-ans_start_tok), answer_end_tok)
    middle_bound = max(ans_start_tok-(answer_end_tok-ans_start_tok),0)

    rand_candidates = list(range(middle_bound))+list(range(answer_end_tok,max_bound))

    rand_start_idx = random.choice(rand_candidates)
    rand_end_idx = rand_start_idx + (answer_end_tok-ans_start_tok)
    return rand_start_idx, rand_end_idx

masked_spans=[]

def find_and_mask(answers, txt):
    masked_txt = txt
    masked=False
    for ans in answers:
        ans_len_char = len(ans)
        ans = re.sub(r'([^a-zA-Z0-9,\.!\? -])',r'\\\1',ans)

        ans_split = ans.split(' ')
        ans_len_token = len(ans_split)

        re_ans = '( )?'.join(ans_split)
        re_ans = re.compile(re_ans,re.I)
        start_idxs = list(re.finditer(re_ans, masked_txt))
        ans_spans = [(-1,0)] + [(start_idx.start(), start_idx.start() + ans_len_char) for start_idx in start_idxs] + [(len(masked_txt),-1)]

        if len(start_idxs) > 0:
            new_masked_txt = ''
            for i, _ in enumerate(ans_spans[:-1]):
                s_idx = ans_spans[i][1]
                e_idx = ans_spans[i+1][0]
                new_masked_txt += masked_txt[s_idx:e_idx]+' '
            masked_txt = new_masked_txt
            masked=True

    if masked:
        return masked_txt
    return None

In [14]:
import spacy
nlp = spacy.load("en_core_web_sm")
#nlp.remove_pipe('sentencizer')
config = {
    'punct_chars':[
        '!', '.', '?', '։', '؟', '۔', '܀', '܁', '܂', '߹',
        '।', '॥', '၊', '။', '።', '፧', '፨', '᙮', '᜵', '᜶', '᠃', '᠉', '᥄',
        '᥅', '᪨', '᪩', '᪪', '᪫', '᭚', '᭛', '᭞', '᭟', '᰻', '᰼', '᱾', '᱿',
            '‼', '‽', '⁇', '⁈', '⁉', '⸮', '⸼', '꓿', '꘎', '꘏', '꛳', '꛷', '꡶',
            '꡷', '꣎', '꣏', '꤯', '꧈', '꧉', '꩝', '꩞', '꩟', '꫰', '꫱', '꯫', '﹒',
            '﹖', '﹗', '！', '．', '？', '𐩖', '𐩗', '𑁇', '𑁈', '𑂾', '𑂿', '𑃀',
            '𑃁', '𑅁', '𑅂', '𑅃', '𑇅', '𑇆', '𑇍', '𑇞', '𑇟', '𑈸', '𑈹', '𑈻', '𑈼',
            '𑊩', '𑑋', '𑑌', '𑗂', '𑗃', '𑗉', '𑗊', '𑗋', '𑗌', '𑗍', '𑗎', '𑗏', '𑗐',
            '𑗑', '𑗒', '𑗓', '𑗔', '𑗕', '𑗖', '𑗗', '𑙁', '𑙂', '𑜼', '𑜽', '𑜾', '𑩂',
            '𑩃', '𑪛', '𑪜', '𑱁', '𑱂', '𖩮', '𖩯', '𖫵', '𖬷', '𖬸', '𖭄', '𛲟', '𝪈',
            '｡', '。',','
    ],
    'overwrite':True,
}
nlp.add_pipe('sentencizer',config=config)

In [15]:
from html.parser import HTMLParser
class MyHTMLParser(HTMLParser):
    #def init_texts(self):
    #    self.texts=['']
        
    def handle_starttag(self, tag, attrs):
        print("Encountered a start tag:", tag)
        #self.texts.append('')

    def handle_endtag(self, tag, attrs):
        print("Encountered a end tag:", tag)
        #self.texts.append('')

    #def handle_data(self, data):
    #    self.texts[-1]+=data 

    #def get_texts(self):
    #    return self.texts

parser = MyHTMLParser()

### NQ train set

In [10]:
from pathlib import Path
import pickle, json

t5_output_path = # glob expression of T5 output files

all_results = list()
for pp in Path('.').glob(t5_output_path):
    with open(pp, 'rb') as f:
        all_results.extend(pickle.load(f))

In [12]:
train_data_path = # Train datasets in DPR format. See https://github.com/facebookresearch/DPR
with open(train_data_path,'r') as f:
    train_data = json.load(f)

In [13]:
all_results_dict = dict()
original_results_dict = dict()

for entry in all_results:
    ctx_id = entry[0]
    question = entry[1]
    ctx = entry[2]
    sent_id = entry[3]
    confidence_score = entry[4]

    if sent_id >= 0:
        if question not in all_results_dict:
            all_results_dict[question] = list()
        all_results_dict[question].append([question, ctx, sent_id, confidence_score])
    else:
        original_results_dict[question] = [question, ctx, sent_id, confidence_score]

In [23]:
from tqdm import tqdm
from scipy.special import softmax
import numpy as np
import copy

cf_data = list()
spans_with_scores = list()

for entry in tqdm(train_data):
    question = entry['question']
    positive_ctxs = entry['positive_ctxs']
    answers = entry['answers']
    cf_ctxs = list()
    spans_per_que = {
        'question' : question,
        'ctx' : positive_ctxs[0],
        'answers': answers 
    }

    for ctx in positive_ctxs[:1]:
        keep_sentences = list()
        text = web_parse(ctx['text'])
        sentences = nlp(text).sents
        
        #print([s.text for s in sentences])
        targets=[]
        target=[]
        for s_id, sent in enumerate(sentences):
            if len(sent)<=3: # consider more than three words as a sentence
                target.extend([t.text for t in sent])
                if len(target)>6:
                    targets.append(' '.join([token for token in target]))
                    target=[]
                    continue
            else:
                targets.append(' '.join([token.text for token in sent]))
                target=[]

        evi_anno = all_results_dict[question]
        evi_cands = sorted(evi_anno, key=lambda x : x[3], reverse=True)
        top_conf_cand = evi_cands[0]
        top_conf_cand_idx = top_conf_cand[2]
        evi_cand_scores = np.array([ca[3] for ca in evi_cands])
        softmax_evi_cand_scores = softmax(evi_cand_scores).tolist()
        

        try:
            original_score = original_results_dict[question][3]
        except IndexError:
            print(original_results_dict[question])

        cand_sents_list = list()
        for cand, sm_score in zip(evi_cands, softmax_evi_cand_scores):
            cand_sent_idx = cand[2]
            if cand_sent_idx >= len(targets):
                continue
            cand_sent = targets[cand_sent_idx]
            cand_score = cand[3]
            cand_sents_list.append({
                'span' : cand_sent,
                'span_idx' : cand_sent_idx,
                'score' : cand_score,
                'softmax_score' : sm_score,
                'score_diff' : (cand_score - original_score)/original_score * 100
            })
        spans_per_que['ctxs'] = cand_sents_list

        try:
            keep_sentences = targets[:top_conf_cand_idx] + targets[top_conf_cand_idx+1:]
        except:
            print(question)
            print(ctx)
            print(top_conf_cand)

        masked_text = ' '.join(keep_sentences)
        parser.feed(masked_text)

        title = ctx['title']
        if find_and_mask(answers, title):
            title = find_and_mask(answers, title)
        
        cf_ctx = copy.deepcopy(ctx)
        cf_ctx['title'] = title
        cf_ctx['text'] = masked_text
        cf_ctxs.append(cf_ctx)
    
    entry['cf_negative_ctxs'] = cf_ctxs
    cf_data.append(entry)
    spans_with_scores.append(spans_per_que)
    

  0%|          | 0/58880 [00:00<?, ?it/s]

 57%|█████▋    | 33332/58880 [27:05<1:00:29,  7.04it/s]

Encountered a start tag: ref


 86%|████████▌ | 50449/58880 [32:36<06:48, 20.64it/s]  

Encountered a start tag: refher


100%|██████████| 58880/58880 [34:48<00:00, 28.19it/s]


In [ ]:
cf_train_data_path = '' # output train file in DPR format.
with open(train_data_path,'w') as f:
    json.dump(cf_data, f, indent=4)

In [24]:
evidence_annotation_path = '' # json files with evidence annotation for case analysis
with open(evidence_annotation_path,'w') as f:
    json.dump(spans_with_scores, f, indent=4)